# 置信度计算

## 实验组和对照组p-value计算
* 取流量作为样本 实验组样本: $n_{exp} = 400000$ 对照组样本: $n_{control} = 400000$
* 取点击流量作为CTR指标 实验组转化：$ct_{exp} = 40500$ 对照组样本: $ct_{control} = 40000$


### 概念
* p-value：对照组和实验组转化率一致的概率
* p-value < 0.05 说明实验组转化率与对照组转化率不一致且实验组优于对照组
### 采取公式
* 公式如下
$$ z = \frac{(p_{exp} - p_{control})}{\sqrt{\frac{p_{exp}(1-p_{exp})}{n_{exp}} + \frac{p_{control}(1-p_{control})}{n_{control}}}}$$

* 个参数
  * x为曝光数，y为产生的结果数（点击、转化等等），n为天数，则转化率为：
  $$ \frac{\sum_{i=1}^ny}{\sum_{i=1}^nx} $$
  * 实验组转化率 $p_{exp}$ ,对照组组转化率 $p_{control}$
  * 样本数实验组 $n_{exp}$ ,对照组组转化率 $n_{control}$

### 实验组对照组差异Z值计算
* 转化率 实验组转化率：$p_{exp}=ct_{exp}/n_{exp}$ 对照组转化率：$p_{control}=ct_{control}/n_{control}$
$$ z = \frac{(p_{exp} - p_{control})}{\sqrt{\frac{p_{exp}(1-p_{exp})}{n_{exp}} + \frac{p_{control}(1-p_{control})}{n_{control}}}}$$

#### python代码计算

In [1]:
import math
from scipy.stats import norm

def get_z(ct_exp, n_experiment, ct_control, n_control):
    p_experiment = ct_exp / n_experiment
    p_control = ct_control / n_control

    se_experiment = p_experiment * (1 - p_experiment) / n_experiment
    se_control = p_control * (1 - p_control) / n_control
    return (p_experiment - p_control) / math.sqrt(se_experiment + se_control)

z = get_z(40500,400000, 40000,400000)
z

1.858243386368076

In [2]:
# p值
norm.sf(z)

0.0315672296806507

## 实验组和对照组置信区间计算
### 概念
* 置信区间：最终转化率有95%的可能性在该区间内
* 本次采用的置信区间按双边检验做验证
### 采取公式
* 增长的时候，公式如下：
$$ (\overline{p_{experiment}} - \overline{p_{control}}) \pm Z_{1-\frac{α}{2}}·\sqrt{\frac{p_{exp}(1-p_{exp})}{n_{exp}} + \frac{p_{control}(1-p_{control})}{n_{control}}}$$
* 降低的时候，公式如下
$$ (\overline{p_{experiment}} - \overline{p_{control}}) \pm Z_{\frac{α}{2}}·\sqrt{\frac{p_{exp}(1-p_{exp})}{n_{exp}} + \frac{p_{control}(1-p_{control})}{n_{control}}}$$
* 其中α取值为0.05

#### python代码计算

In [3]:
import math
from scipy.stats import norm

def get_confidence_interval(ct_exp, n_experiment, ct_control, n_control):
    p_experiment = ct_exp / n_experiment
    p_control = ct_control / n_control

    se_experiment = p_experiment * (1 - p_experiment) / n_experiment
    se_control = p_control * (1 - p_control) / n_control
    wave = norm.ppf(1-0.05/2) * math.sqrt(se_experiment + se_control)

    return [p_experiment - p_control - wave,p_experiment - p_control + wave]

# 置信区间
get_confidence_interval(40500,400000, 40000,400000)

[-6.842523893680458e-05, 0.002568425238936807]

## 实验组和对照组统计功效计算
### 概念
* 统计功效：默认对照组和实验组不一致，实际计算对照组和实验组不一致的概率

### 采取公式
* 公式如下
$$ Power=Φ(Z_{1-\frac{α}{2}} -  \frac{(p_{exp} - p_{control})}{\sqrt{\frac{p_{exp}(1-p_{exp})}{n_{exp}} \frac{p_{control}(1-p_{control})}{n_{control}}}}) + 1 - Φ(-Z_{1-\frac{α}{2}} -  \frac{(p_{exp} - p_{control})}{\sqrt{\frac{p_{exp}(1-p_{exp})}{n_{exp}} + \frac{p_{control}(1-p_{control})}{n_{control}}}})  $$

In [4]:
import math
from scipy.stats import norm

def get_z(ct_exp, n_experiment, ct_control, n_control):
    p_experiment = ct_exp / n_experiment
    p_control = ct_control / n_control

    se_experiment = p_experiment * (1 - p_experiment) / n_experiment
    se_control = p_control * (1 - p_control) / n_control
    return (p_experiment - p_control) / math.sqrt(se_experiment + se_control)

z = get_z(40500,400000,40000,400000)
# 统计功效
norm.sf(norm.ppf(1-0.05/2) - z) + 1 - norm.sf(-z - norm.ppf(1-0.05/2))

0.45955643844271055